In [1]:
import os
os.getcwd()

'H:\\Master-Indoors-Project\\Magnetics\\scripts'

In [2]:
import os
import sys
import numpy as np
module_path = os.path.abspath(os.path.join('..','src'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from data_labels import APARTMENT_WALKS_BAD, APARTMENT_WALKS_IBO, APARTMENT_WALKS_RICCI
from sensor import SensorData
import matplotlib.pyplot as plt

In [3]:
#setup the datasets to be used and the containers for processed data
data_sets = [APARTMENT_WALKS_BAD,APARTMENT_WALKS_RICCI,APARTMENT_WALKS_IBO]
trainings = []
tests = []

In [4]:
#test for Bath
bath_folder = APARTMENT_WALKS_BAD
bath_files = os.listdir(bath_folder)
bath_test = []
bath_training = []
get_test = True

In [5]:
for csv_file in bath_files:
    name,ext = os.path.splitext(csv_file)
    if ext != '.csv':
        continue
    print name
    csv_file = os.path.join(bath_folder,csv_file)
    sensordata = SensorData(csv_file)
    if get_test:
        bath_test.append(sensordata.magnetic())
        get_test = False
    else:
        bath_training.append(sensordata.magnetic())

Sensor_record_20160912_092941_AndroSensor
Sensor_record_20160912_093016_AndroSensor
Sensor_record_20160912_093055_AndroSensor
Sensor_record_20160912_093123_AndroSensor
Sensor_record_20160912_093159_AndroSensor
Sensor_record_20160912_093230_AndroSensor


In [6]:
bath_test[0].head(), bath_training[0].head()

(                         MAGNETIC FIELD X (μT)  MAGNETIC FIELD Y (μT)  \
 YYYY-MO-DD HH-MI-SS_SSS                                                 
 2016-09-12 09:29:26.131                -18.375                 -3.020   
 2016-09-12 09:29:26.631                -19.147                 -1.926   
 2016-09-12 09:29:27.131                -19.753                 -2.582   
 2016-09-12 09:29:27.631                -17.224                 -4.562   
 2016-09-12 09:29:28.132                -22.020                 -0.624   
 
                          MAGNETIC FIELD Z (μT)  
 YYYY-MO-DD HH-MI-SS_SSS                         
 2016-09-12 09:29:26.131                -42.883  
 2016-09-12 09:29:26.631                -42.274  
 2016-09-12 09:29:27.131                -41.801  
 2016-09-12 09:29:27.631                -43.388  
 2016-09-12 09:29:28.132                -39.372  ,
                          MAGNETIC FIELD X (μT)  MAGNETIC FIELD Y (μT)  \
 YYYY-MO-DD HH-MI-SS_SSS                               

In [7]:
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

In [8]:
for i in range(3):
    distance, path = fastdtw(bath_test[0].iloc[:,i],  bath_training[0].iloc[:,i], dist=euclidean)
    print i,distance

0 46.46
1 72.529
2 34.355


In [9]:
from matplotlib import collections  as mc
def dtw_plot(x, y, path, distance):
    fig, axs = plt.subplots(1, 3)


    axs[0].plot(x[:, 0], x[:, 1], color="green", label='x')
    axs[0].plot(y[:, 0], y[:, 1], color="red", label='y')


    axs[1].scatter(x=np.array(path)[:, 0], y=np.array(path)[:, 1], label='dtw:path')

    # create lines using the paths data and the dataframes
    lines = [[x[p1], y[p2]] for p1, p2 in zip(np.array(path)[:, 0], np.array(path)[:, 1])]
    # add to line collection
    lc = mc.LineCollection(lines, linewidths=2, label='dtw:shortest')
    # plot the line collection
    axs[2].add_collection(lc)
    axs[2].autoscale()
    axs[2].margins(0.1)
    axs[2].plot(x[:, 0], x[:, 1], color="green", label='x')
    axs[2].plot(y[:, 0], y[:, 1], color="red", label='y')

    # show all legends
    for ax in axs:
        ax.legend()

    plt.suptitle("DTW - TEST \n Distance:%f"%distance)

    return fig, axs, plt

In [10]:
for i in range(3):
    x = np.array(zip(bath_test[0].iloc[:,i].index.to_series().diff().dt.total_seconds().fillna(0).cumsum().values,bath_test[0].iloc[:,i]))
    y = np.array(zip(bath_training[0].iloc[:,i].index.to_series().diff().dt.total_seconds().fillna(0).cumsum().values,bath_training[0].iloc[:,i]))
    distance, path = fastdtw(x, y, dist=euclidean)
    print distance
    #fig, axs, done_plt = dtw_plot(x, y, path, distance)
    #done_plt.show()

57.9633790819
85.9253328167
45.5651955698
